In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# --- Load CIFAR-10 ---
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train.astype("float32") / 255.0, x_test.astype("float32") / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# --- Simple CNN + Transformer ---
inputs = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2))(x)

# Flatten spatial dims for transformer
x = layers.Reshape((8*8, 64))(x)  # (batch, tokens, features)

# Transformer block
x = layers.LayerNormalization()(x)
attn_output = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = layers.Add()([x, attn_output])

x = layers.LayerNormalization()(x)
ff = layers.Dense(128, activation='relu')(x)
ff = layers.Dense(64)(ff)  # match dimensions
x = layers.Add()([x, ff])

# Classification head
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Train ---
history = model.fit(x_train, y_train, epochs=4, batch_size=64,
                    validation_data=(x_test, y_test))

# --- Evaluate ---
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\n✅ CIFAR-10 Test Accuracy: {test_acc:.4f}")

